# RPFR 3D Visualization

Interactive 3D visualization of per-atom RPFR values using **RDKit** (molecule structure) and **py3Dmol** (3D rendering).

**Test molecule:** Methyl carbamate — `COC(=O)NC(N)=O`  
Contains C, H, N, and O atoms → good demonstration of the cross-element scale problem.

---
### The cross-element RPFR problem

RPFR values are **not comparable across elements**:

| Element | Typical RPFR at 300 K |
|---------|----------------------|
| H | 10 – 15 |
| C | 1.10 – 1.25 |
| N | 1.05 – 1.15 |
| O | 1.08 – 1.15 |

Hydrogen exchange involves a much larger mass ratio (²H/¹H ≈ 2) compared to heavy-atom isotopes (¹³C/¹²C ≈ 1.08), so H RPFR is physically on a different scale.

This notebook demonstrates four visualization strategies to handle this.

In [1]:
import sys
from pathlib import Path
import numpy as np
import h5py

# Path setup
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'src'))

from rpfr_gui.data import ChemistryResolver, H5Provider
from rpfr_gui.domain import IsotopeGraph
from rpfr_gui.ui import RPFRVisualizer

H5_PATH  = Path('/Users/simonandren/QM9_H5/qm9s.h5')
IDX_PATH = project_root / 'data/processed/index.parquet'

print('Libraries loaded OK')

Libraries loaded OK


## 1. Load the test molecule

In [2]:
SMILES = 'COC(=O)NC(N)=O'   # Methyl carbamate
TEMPERATURE = 300.0

resolver = ChemistryResolver(IDX_PATH)
provider = H5Provider(H5_PATH)

mol_id = resolver.resolve(SMILES, id_type='smiles')
print(f'Molecule ID: {mol_id}')

# Load per-atom data from HDF5 (DFT-optimised coords + RPFR)
with h5py.File(H5_PATH, 'r') as f:
    g = f[mol_id]
    symbols = [s.decode() for s in g['Atom_Symbol'][()]]
    coords  = np.stack([g['x'][()], g['y'][()], g['z'][()]], axis=1)
    rpfr    = g[f'RPFR_{int(TEMPERATURE)}K'][()]

import pandas as pd
df = pd.DataFrame({'atom': symbols, 'x': coords[:,0], 'y': coords[:,1], 'z': coords[:,2], 'RPFR': rpfr})
print(f'\n{len(df)} atoms loaded')
print(df.groupby('atom')['RPFR'].describe().round(5))

Molecule ID: 005752

14 atoms loaded
      count      mean      std       min       25%       50%       75%  \
atom                                                                     
C       3.0   1.18068  0.03514   1.14014   1.16988   1.19962   1.20095   
H       6.0  14.17134  1.02189  13.19008  13.35903  13.92393  14.78134   
N       2.0   1.10352  0.01264   1.09458   1.09905   1.10352   1.10799   
O       3.0   1.12027  0.00573   1.11567   1.11707   1.11847   1.12258   

           max  
atom            
C      1.20229  
H     15.75225  
N      1.11246  
O      1.12669  


## 2. Element-constrained normalization

Isotope exchange only occurs within the same element (C↔C, H↔H, etc.).  
Setting a **carbon** anchor and asking for **hydrogen** relative RPFR is physically undefined — the code now returns `NaN` for cross-element queries.

In [3]:
rpfr_df = provider.get_rpfr(mol_id, temperature=TEMPERATURE)

graph = IsotopeGraph(connectivity='full')
nodes = graph.add_molecule(mol_id, rpfr_df)
graph.set_connectivity(mode='full')

# Set the carbonyl carbon (atom 2, C in C=O of ester) as anchor
c_nodes = [n for n in nodes if graph.graph.nodes[n]['atom_symbol'] == 'C']
graph.set_anchor(c_nodes[0])
print(f'Anchor: {c_nodes[0]}  element: {graph.anchor_element}')

df_rel = graph.get_rpfr_dataframe(relative=True)
print('\nRelative RPFR (NaN = different element from anchor):')
display(df_rel[['atom_symbol','rpfr' if 'rpfr' in df_rel.columns else 'relative_rpfr']]
        .rename(columns={'relative_rpfr': 'rel_rpfr'}))

Anchor: 005752_0  element: C

Relative RPFR (NaN = different element from anchor):


,atom_symbol,rel_rpfr
0,C,1.000000
1,O,NaN
2,C,1.054513
3,O,NaN
4,N,NaN
5,C,1.052171
6,N,NaN
7,O,NaN
8,H,NaN
9,H,NaN


In [4]:
# Per-element normalized view (minmax)
df_norm = graph.get_element_normalized_rpfr(method='minmax')
print('Minmax per element (0 = lowest in element, 1 = highest in element):')
display(df_norm[['node_id', 'atom_symbol', 'rpfr', 'element_mean', 'normalized_rpfr']].round(4))

Minmax per element (0 = lowest in element, 1 = highest in element):


,node_id,atom_symbol,rpfr,element_mean,normalized_rpfr
0,005752_0,C,1.1401,1.1807,0.0000
1,005752_1,O,1.1267,1.1203,1.0000
2,005752_2,C,1.2023,1.1807,1.0000
3,005752_3,O,1.1185,1.1203,0.2544
4,005752_4,N,1.1125,1.1035,1.0000
5,005752_5,C,1.1996,1.1807,0.9570
6,005752_6,N,1.0946,1.1035,0.0000
7,005752_7,O,1.1157,1.1203,0.0000
8,005752_8,H,13.7154,14.1713,0.2050
9,005752_9,H,13.1901,14.1713,0.0000


## 3. Build the visualizer

All four modes use the **DFT-optimised 3D coordinates** from the HDF5 file (not a generated conformer).

In [5]:
viz = RPFRVisualizer(SMILES, symbols, coords, rpfr, temperature=TEMPERATURE)
print(f'Molecule: {SMILES}')
print(f'Atoms: {viz.n_atoms}  |  Elements: {viz._elements}  |  T = {TEMPERATURE:.0f} K')
print()
display(viz.summary_table().round(4))

Molecule: COC(=O)NC(N)=O
Atoms: 14  |  Elements: ['C', 'H', 'N', 'O']  |  T = 300 K



,idx,symbol,rpfr,el_mean,el_min,el_max,minmax
0,0,C,1.1401,1.1807,1.1401,1.2023,0.0000
1,1,O,1.1267,1.1203,1.1157,1.1267,1.0000
2,2,C,1.2023,1.1807,1.1401,1.2023,1.0000
3,3,O,1.1185,1.1203,1.1157,1.1267,0.2544
4,4,N,1.1125,1.1035,1.0946,1.1125,1.0000
5,5,C,1.1996,1.1807,1.1401,1.2023,0.9570
6,6,N,1.0946,1.1035,1.0946,1.1125,0.0000
7,7,O,1.1157,1.1203,1.1157,1.1267,0.0000
8,8,H,13.7154,14.1713,13.1901,15.7523,0.2050
9,9,H,13.1901,14.1713,13.1901,15.7523,0.0000


## Mode 1 — Element filter

**Scientifically strictest.** Display only atoms of one element at a time,  
coloured on their own min-max RPFR scale.  
Other atoms shown as faint grey sticks for structural context.

Use this when you want to compare site-specific fractionation within one element.

In [6]:
# Hydrogen sites — largest absolute RPFR variation
viz.show_element_filter('C')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
# Carbon sites
viz.show_element_filter('O')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
# Oxygen sites
viz.show_element_filter('N')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Mode 2 — Multi-element (independent colour scales)

All atoms shown simultaneously, each element with its own colormap:  
- **H** → Blues  
- **C** → Greens  
- **N** → Purples  
- **O** → Oranges  

Each element is independently min-max normalised so variation within each element is visible.

**⚠️ Note:** Colours are NOT comparable across elements — a dark blue H is not the same fractionation level as a dark green C.

In [9]:
viz.show_multi_element()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Mode 3 — Global log scale

All atoms on one shared colour scale using **log₁₀(RPFR)**.  
Log scale is essential here — without it, H atoms dominate and all heavy atoms look the same.

The H atoms will still appear at the yellow/bright end and C/N/O at the blue/dark end,  
but internal variation within both groups becomes visible.

Use this for a raw first look at the full dataset.

In [10]:
viz.show_global_scale(log_scale=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
# Linear scale for comparison — heavy atoms become indistinguishable
viz.show_global_scale(log_scale=False, cmap='plasma')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Mode 3 — Global (log) scale

**Useful for a raw overview of all elements at once.**  
Because H RPFR (~10–15) is ~10× larger than C/N/O (~1.1–1.2),
a **log₁₀ scale** is strongly recommended so variation within heavy atoms is still visible.


In [12]:
viz.show_global_scale()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [13]:
# Linear scale (less useful — hydrogen dominates)
viz.show_global_scale(log_scale=False)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 5. Compare molecules: benzene vs. methane

In [14]:
def load_and_visualize(smiles, temperature=300.0):
    """Helper: resolve, load, and return a RPFRVisualizer."""
    mol_id = resolver.resolve(smiles, id_type='smiles')
    if mol_id is None:
        print(f'Not found: {smiles}'); return None
    with h5py.File(H5_PATH, 'r') as f:
        g = f[mol_id]
        syms  = [s.decode() for s in g['Atom_Symbol'][()]]
        xyz   = np.stack([g['x'][()], g['y'][()], g['z'][()]], axis=1)
        rpfr  = g[f'RPFR_{int(temperature)}K'][()]
    print(f'{smiles} ({mol_id}) — {len(syms)} atoms')
    return RPFRVisualizer(smiles, syms, xyz, rpfr, temperature=temperature)

viz_benzene = load_and_visualize('c1ccccc1')
viz_methane = load_and_visualize('C')

c1ccccc1 (000197) — 12 atoms
C (000001) — 5 atoms


In [15]:
print('Benzene — per-element minmax (all H are equivalent by symmetry):')
viz_benzene.show_multi_element()

Benzene — per-element minmax (all H are equivalent by symmetry):


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [16]:
print('Methane (all H equivalent):')
viz_methane.show_multi_element()

Methane (all H equivalent):


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 6. Temperature dependence

RPFR values are temperature-dependent. Available temperatures in qm9s.h5:  
`20, 60, 100, 140, 180, 220, 260, 273, 298, 300, 340, 380, 420 K`

In [17]:
from ipywidgets import interact, Dropdown

SMILES_MCC = 'COC(=O)NC(N)=O'   # Methyl carbamate

SMILES_benzene = 'c1ccccc1'   # Benzene

TEMPS = [20, 60, 100, 140, 180, 220, 260, 273, 298, 300, 340, 380, 420]
MODES = ['element_filter_H', 'multi_element', 'global_scale']

def show_at_temperature(temperature=300, mode='multi_element', element='H'):
    v = load_and_visualize(SMILES_MCC, temperature=float(temperature))
    if mode == 'element_filter_H':
        display(v.show_element_filter(element))
    elif mode == 'multi_element':
        display(v.show_multi_element())
    elif mode == 'global_scale':
        display(v.show_global_scale())

interact(
    show_at_temperature,
    temperature=Dropdown(options=TEMPS, value=300),
    mode=Dropdown(options=MODES, value='multi_element'),
    element=Dropdown(options=['H', 'C', 'N', 'O'], value='H'),
)

interactive(children=(Dropdown(description='temperature', index=9, options=(20, 60, 100, 140, 180, 220, 260, 2…

<function __main__.show_at_temperature(temperature=300, mode='multi_element', element='H')>